In [1]:
# coding=utf8

import os
import json
import numpy as np
import pandas as pd
from pprint import pprint
from typing import List, Dict
from sql_hardness import eval_hardness, eval_hardness_by_from

In [14]:
original_sparc_dev_path = os.path.join('..', 'data', 'original_sparc_dev.json')
sparc_dev_annotated_path = os.path.join('..', 'data', 'sparc_dev_annotated.tsv')
duorat_sparc_path = os.path.join('..', 'data', 'duorat_sparc.json')
editsql_sparc_path = os.path.join('..', 'data', 'editsql_sparc.json')
igsql_sparc_path = os.path.join('..', 'data', 'igsql_sparc.json')

In [38]:
def read_annotation(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep='\t')
    df.fillna('', inplace=True)
    return df


def read_json(path: str) -> List[Dict]:
    with open(path, 'r', encoding="utf8") as f:
        return json.load(f)


def read_predictions(path: str) -> List[Dict]:
    predictions = list()
    with open(path, 'r') as f:
        predictions = json.load(f)
    interaction_ids, turn_ids = set(), set()
    for p in predictions:
        turn_ids.add(p['turn_id'])
        interaction_ids.add(p['interaction_id'])
    total_question_sequence = len(interaction_ids)
    total_question = len(predictions)
    print(total_question_sequence, total_question)
    
    grouped_predictions = list()
    for example_id in interaction_ids:
        result = {"example_id": example_id, "interaction": list()}
        for turn_id in [1, 2, 3, 4, 5]:
            for p in predictions:
                if p['interaction_id'] == example_id and p['turn_id'] == turn_id:
                    result['interaction'].append(p)
                    break
            else:
                # Not found
                break
        grouped_predictions.append(result)
    
    print(len(grouped_predictions))
    assert len(grouped_predictions) == total_question_sequence
    _count = sum([len(gp['interaction']) for gp in grouped_predictions])
    print(_count)
    assert _count == total_question
    
    return grouped_predictions


def measure_accuracy(predictions: List[Dict]):
    question_match = 0
    question_sequence_match = 0
    total_question = sum([len(p['interaction']) for p in predictions])
    total_question_sequence = len(predictions)
    
    for p in predictions:
        has_error = False
        for turn in p['interaction']:
            if turn['exact'] == 1 or turn['exact'] is True:
                question_match += 1
            else:
                has_error = True
        if not has_error:
            question_sequence_match += 1
    
    print("Question Match: %f" % (question_match / total_question))
    print("Interaction Match: %f" % (question_sequence_match / total_question_sequence))
    

CONTEXTUAL_PHENOMENA = ['Coreference', 'Part/Whole', 'Ellipsis', 'Explict/Implict', 'Operator/Schema', 'Modification/Continutation']

    
def measure_context_independent_accuracy(predictions: List[Dict], source_examples: List[Dict], annotation: pd.DataFrame):
    dependent_match, dependent_total = 0, 0
    independent_match, independent_total = 0, 0
    first_independent_match, first_independent_total = 0, 0
    other_independent_match, other_independent_total = 0, 0
    total_question = 0
    independent_hardness = {"easy": 0, "medium": 0, "hard": 0, "extra": 0}
    dependent_hardness = {"easy": 0, "medium": 0, "hard": 0, "extra": 0}
    independent_hardness_match = {"easy": 0, "medium": 0, "hard": 0, "extra": 0}
    dependent_hardness_match = {"easy": 0, "medium": 0, "hard": 0, "extra": 0}

    for p in predictions:
        example_id = p['example_id']
        for turn in p['interaction']:
            total_question += 1
            turn_id = turn['turn_id']
            source_turn = source_examples[example_id - 1]['interaction'][turn_id - 1]
            print(source_turn['utterance'].lower().strip(), turn['question'].lower().strip())
            assert source_turn['utterance'].lower().strip().replace(" ", "").replace("`", "").replace('"', "").replace("''", "") in turn['question'].lower().strip().replace(" ", "").replace("`", "").replace('"', "").replace("''", "")
            hardness = eval_hardness(source_turn['sql'])
            row = annotation[(annotation.InterID == example_id - 1) & (annotation.Turn == turn_id - 1)]
            is_independent = all([row.iloc[0][key] == "" for key in CONTEXTUAL_PHENOMENA]) or row.iloc[0]["Ellipsis"] == "Context Independent"
            if is_independent:
                if turn['exact'] == 1 or turn['exact'] is True:
                    independent_match += 1
                    independent_hardness_match[hardness] += 1
                independent_total += 1
                independent_hardness[hardness] += 1
                
                if row.iloc[0]["Ellipsis"] == "Context Independent":
                    if turn['exact'] == 1 or turn['exact'] is True:
                        other_independent_match += 1
                    other_independent_total += 1
                else:
                    # First
                    if turn['exact'] == 1 or turn['exact'] is True:
                        first_independent_match += 1
                    first_independent_total += 1
                
            else:
                if turn['exact'] == 1 or turn['exact'] is True:
                    dependent_match += 1
                    dependent_hardness_match[hardness] += 1
                dependent_total += 1
                dependent_hardness[hardness] += 1
    
    print("Independent: %d, %d, %f" % (independent_match, independent_total, independent_match / independent_total))
    print("Dependent: %d, %d, %f" % (dependent_match, dependent_total, dependent_match / dependent_total))
    
    print("First Independent: %d, %d, %f" % (first_independent_match, first_independent_total, first_independent_match / first_independent_total))
    print("Other Independent: %d, %d, %f" % (other_independent_match, other_independent_total, other_independent_match / other_independent_total))
    
    assert independent_total + dependent_total == total_question
    print("Independent Percent: %f" % (independent_total / total_question))

    pprint(independent_hardness)
    pprint(independent_hardness_match)
    pprint(dependent_hardness)

    pprint({key: value / independent_total for key, value in independent_hardness.items()})
    pprint({key: value / dependent_total for key, value in dependent_hardness.items()})
    
    # Total Hardness
    total_hardness = {k: independent_hardness[k] + dependent_hardness[k] for k in ["easy", "medium", "hard", "extra"]}
    total_hardness_match = {k: independent_hardness_match[k] + dependent_hardness_match[k] for k in ["easy", "medium", "hard", "extra"]}
    total_hardness = {k: independent_hardness[k] + dependent_hardness[k] for k in ["easy", "medium", "hard", "extra"]}
    total_hardness_distribution = {k: (independent_hardness[k] + dependent_hardness[k]) / total_question for k in ["easy", "medium", "hard", "extra"]}
    total_hardness_accuracy = {k: total_hardness_match[k] / total_hardness[k] for k in ["easy", "medium", "hard", "extra"]}
    print("Total Hardness: ", total_hardness)
    print("Total Hardness Distribution: ", total_hardness_distribution)
    print("Total Hardness Match: ", total_hardness_match)
    print("Total Hardness Accuracy: ", total_hardness_accuracy)

In [4]:
sparc_dev = read_json(original_sparc_dev_path)

In [5]:
duorat_sparc_predictions = read_predictions(duorat_sparc_path)

422 1203
422
1203


In [6]:
measure_accuracy(duorat_sparc_predictions)

Question Match: 0.600998
Interaction Match: 0.386256


In [7]:
sparc_annotation_df = read_annotation(sparc_dev_annotated_path)

In [8]:
sparc_annotation_df[(sparc_annotation_df.InterID == 0) & (sparc_annotation_df.Turn == 0)]

,InterID,Turn,Database_ID,Utterance,Coreference,Part/Whole,Ellipsis,Explict/Implict,Operator/Schema,Modification/Continutation
0,0,0,flight_2,what are all the airlines ?,,,,,,


In [32]:
measure_context_independent_accuracy(duorat_sparc_predictions, sparc_dev, sparc_annotation_df)

what are all the airlines? [sep] what are all the airlines?
of these, which is jetblue airways? [sep] of these, which is jetblue airways? [sep] what are all the airlines?
what is the country corresponding it? [sep] what is the country corresponding it? [sep] of these, which is jetblue airways? [sep] what are all the airlines?
what are all the abbreviations? [sep] what are all the abbreviations?
what is the abbreviation for jetblue airways? [sep] what is the abbreviation for jetblue airways? [sep] what are all the abbreviations?
list all airline names and their abbreviations. [sep] list all airline names and their abbreviations.
of these, list only those in the usa. [sep] of these, list only those in the usa. [sep] list all airline names and their abbreviations.
find all information about the airport in the city anthony. [sep] find all information about the airport in the city anthony.
what is the airport code and airport name? [sep] what is the airport code and airport name? [sep] find

what are their first names and ages? [sep] what are their first names and ages? [sep] of those, who does not own a cat? [sep] which students have dogs?
who is the youngest pet? [sep] who is the youngest pet?
what type are they? [sep] what type are they? [sep] who is the youngest pet?
also, how much do they weigh? [sep] also, how much do they weigh? [sep] what type are they? [sep] who is the youngest pet?
what are ids of every pet? [sep] what are ids of every pet?
for those, which refer to pets older than 1? [sep] for those, which refer to pets older than 1? [sep] what are ids of every pet?
also, how much do they weigh? [sep] also, how much do they weigh? [sep] for those, which refer to pets older than 1? [sep] what are ids of every pet?
what are the different pet types? [sep] what are the different pet types?
for each of those, what is the maximum age? [sep] for each of those, what is the maximum age? [sep] what are the different pet types?
what about the average age? [sep] what about 

of those countries, give the country codes for those which do not speak english. [sep] of those countries, give the country codes for those which do not speak english. [sep] what are the country codes of all the countries?
what are the country codes of all the countries? [sep] what are the country codes of all the countries?
of those, which do not speak english? [sep] of those, which do not speak english? [sep] what are the country codes of all the countries?
what are the country codes for countries which are not republics? [sep] what are the country codes for countries which are not republics?
of those, which do not speak english? [sep] of those, which do not speak english? [sep] what are the country codes for countries which are not republics?
what are the names of the countries where english is the official language? [sep] what are the names of the countries where english is the official language?
what are the names of the countries in europe that are not one of those? [sep] what ar

group people by their nationality. [sep] group people by their nationality. [sep] show the nationality of each person.
for each of the result, show the corresponding number of people of that nationality? [sep] for each of the result, show the corresponding number of people of that nationality? [sep] group people by their nationality. [sep] show the nationality of each person.
show the nationality of each person. [sep] show the nationality of each person.
count the number of people for each nationality. [sep] count the number of people for each nationality. [sep] show the nationality of each person.
what is the most common nationality of people? [sep] what is the most common nationality of people? [sep] count the number of people for each nationality. [sep] show the nationality of each person.
count the number of people for each nationality. [sep] count the number of people for each nationality.
which nationalities are held by at least two people? [sep] which nationalities are held by a

list just the full name and id! [sep] list just the full name and id! [sep] how about more than 3? [sep] which ones have designed more than 1? [sep] what is all the information for the makers that have designed at least 1 car model!
how many different models has maker with full name general motors produced? [sep] how many different models has maker with full name general motors produced?
how many different models weigh over 3500 pounds? [sep] how many different models weigh over 3500 pounds? [sep] how many different models has maker with full name general motors produced?
how many models have either of these features? [sep] how many models have either of these features? [sep] how many different models weigh over 3500 pounds? [sep] how many different models has maker with full name general motors produced?
show me the distinct list of their model names! [sep] show me the distinct list of their model names! [sep] how many models have either of these features? [sep] how many different mod

...

show the id for all paragraphs. [sep] show the id for all paragraphs.
also show their document names. [sep] also show their document names. [sep] show the id for all paragraphs.
only show the id for all paragraphs in the document named 'summer show'. [sep] only show the id for all paragraphs in the document named 'summer show'. [sep] also show their document names. [sep] show the id for all paragraphs.
how many such paragraphs are there? [sep] how many such paragraphs are there? [sep] only show the id for all paragraphs in the document named 'summer show'. [sep] also show their document names. [sep] show the id for all paragraphs.
show the parapraph text for all paragraphs. [sep] show the parapraph text for all paragraphs.
also show their paragraph details. [sep] also show their paragraph details. [sep] show the parapraph text for all paragraphs.
what are the details for the paragraph containing 'korea'? [sep] what are the details for the paragraph containing 'korea'? [sep] also show t

which cities did they come from? [sep] which cities did they come from? [sep] find all employees who are under age 30.
show the cities from which more than one employee originated. [sep] show the cities from which more than one employee originated. [sep] which cities did they come from? [sep] find all employees who are under age 30.
show all shops. [sep] show all shops.
what is the total number of products they are selling? [sep] what is the total number of products they are selling? [sep] show all shops.
find the name of the shop with the greatest number of products. [sep] find the name of the shop with the greatest number of products. [sep] what is the total number of products they are selling? [sep] show all shops.
what are its manager name and district? [sep] what are its manager name and district? [sep] find the name of the shop with the greatest number of products. [sep] what is the total number of products they are selling? [sep] show all shops.
find the average number of produc

how many people do they each like? [sep] how many people do they each like? [sep] how about those who like others? [sep] what are the names of the highschoolers who don't like anyone?
how many high schoolers like somebody else? [sep] how many high schoolers like somebody else?
what are their names? [sep] what are their names? [sep] how many high schoolers like somebody else?
what is the greatest number of likes anyone has? [sep] what is the greatest number of likes anyone has? [sep] what are their names? [sep] how many high schoolers like somebody else?
who has the greatest number of likes? [sep] who has the greatest number of likes? [sep] what is the greatest number of likes anyone has? [sep] what are their names? [sep] how many high schoolers like somebody else?
show me the names of all students? [sep] show me the names of all students?
how about those who like at least one other person! [sep] how about those who like at least one other person! [sep] show me the names of all students

show me just the top 1. [sep] show me just the top 1. [sep] show me the id, first and last names of top 3 students who have enrolled for the most times in any program.
give me the description of a semester with no enrolled students. [sep] give me the description of a semester with no enrolled students.
what's the name of that semester? [sep] what's the name of that semester? [sep] give me the description of a semester with no enrolled students.
what's the name of the course with the least number of enrollments? [sep] what's the name of the course with the least number of enrollments?
how many enrollments does the course with most enrollments have? [sep] how many enrollments does the course with most enrollments have? [sep] what's the name of the course with the least number of enrollments?
what's the name of that course? [sep] what's the name of that course? [sep] how many enrollments does the course with most enrollments have? [sep] what's the name of the course with the least number 

which one is the most recent cost? [sep] which one is the most recent cost? [sep] what about in descending order? [sep] order the cost in ascending order. [sep] show the treatment details.
show all the owner information. [sep] show all the owner information.
what about the dogs? [sep] what about the dogs? [sep] show all the owner information.
among those, how many of them have been sent to receive treatment? [sep] among those, how many of them have been sent to receive treatment? [sep] what about the dogs? [sep] show all the owner information.
what about those who did not? [sep] what about those who did not? [sep] among those, how many of them have been sent to receive treatment? [sep] what about the dogs? [sep] show all the owner information.
show all the owner information. [sep] show all the owner information.
how many of them come from the state of 'arizona'? [sep] how many of them come from the state of 'arizona'? [sep] show all the owner information.
what about those who do not ha

In [10]:
(263 + 50 + 35) / (441 + 145 + 134)

0.48333333333333334

In [11]:
483 + 441 + 145 + 134

1203

### IGSQL

In [23]:
igsql_sparc_predictions = read_predictions(igsql_sparc_path)

422 1203
422
1203


In [33]:
igsql_sparc_predictions[0]

{'example_id': 1,
 'interaction': [{'interaction_id': 1,
   'turn_id': 1,
   'question': 'What are all the airlines ?',
   'ground_truth': 'SELECT * FROM AIRLINES',
   'sql': 'select Airline from airlines',
   'exact': 0},
  {'interaction_id': 1,
   'turn_id': 2,
   'question': 'Of these , which is Jetblue Airways ?',
   'ground_truth': 'SELECT * FROM AIRLINES WHERE Airline  =  "JetBlue Airways"',
   'sql': 'select Airline from airlines where Airline = 1',
   'exact': 0},
  {'interaction_id': 1,
   'turn_id': 3,
   'question': 'What is the country corresponding it ?',
   'ground_truth': 'SELECT Country FROM AIRLINES WHERE Airline  =  "JetBlue Airways"',
   'sql': 'select Country from airlines where Airline = 1',
   'exact': True}]}

In [21]:
measure_accuracy(igsql_sparc_predictions)

Question Match: 0.495428
Interaction Match: 0.300948


In [39]:
measure_context_independent_accuracy(igsql_sparc_predictions, sparc_dev, sparc_annotation_df)

what are all the airlines? what are all the airlines ?
of these, which is jetblue airways? of these , which is jetblue airways ?
what is the country corresponding it? what is the country corresponding it ?
what are all the abbreviations? what are all the abbreviations ?
what is the abbreviation for jetblue airways? what is the abbreviation for jetblue airways ?
list all airline names and their abbreviations. list all airline names and their abbreviations .
of these, list only those in the usa. of these , list only those in the usa .
find all information about the airport in the city anthony. find all information about the airport in the city anthony .
what is the airport code and airport name? what is the airport code and airport name ?
what are all the airlines? what are all the airlines ?
how many are there? how many are there ?
what are all the airports? what are all the airports ?
how many are there? how many are there ?
what are all the flights? what are all the flights ?
how many

list all the other ids. list all the other ids .
which students have dogs? which students have dogs ?
of those, who does not own a cat? of those , who does not own a cat ?
what are their first names and ages? what are their first names and ages ?
who is the youngest pet? who is the youngest pet ?
what type are they? what type are they ?
also, how much do they weigh? also , how much do they weigh ?
what are ids of every pet? what are ids of every pet ?
for those, which refer to pets older than 1? for those , which refer to pets older than 1 ?
also, how much do they weigh? also , how much do they weigh ?
what are the different pet types? what are the different pet types ?
for each of those, what is the maximum age? for each of those , what is the maximum age ?
what about the average age? what about the average age ?
what information is there on pets? what information is there on pets ?
for each pet type, what is the average weight? for each pet type , what is the average weight ?
which s

of those countries, give the country codes for those which do not speak english. of those countries , give the country codes for those which do not speak english .
what are the country codes of all the countries? what are the country codes of all the countries ?
of those, which do not speak english? of those , which do not speak english ?
what are the country codes for countries which are not republics? what are the country codes for countries which are not republics ?
of those, which do not speak english? of those , which do not speak english ?
what are the names of the countries where english is the official language? what are the names of the countries where english is the official language ?
what are the names of the countries in europe that are not one of those? what are the names of the countries in europe that are not one of those ?
what are the names of the cities in those countries? what are the names of the cities in those countries ?
what are the countries in asia where chin

for each of the result, show the corresponding number of people of that nationality? for each of the result , show the corresponding number of people of that nationality ?
show the nationality of each person. show the nationality of each person .
count the number of people for each nationality. count the number of people for each nationality .
what is the most common nationality of people? what is the most common nationality of people ?
count the number of people for each nationality. count the number of people for each nationality .
which nationalities are held by at least two people? which nationalities are held by at least two people ?
what are the names and birth dates of people? what are the names and birth dates of people ?
sort the results in ascending alphabetical order of name. sort the results in ascending alphabetical order of name .
what are the names of all people? what are the names of all people ?
what about people whose nationality is not "russia"? what about people who

how many different models weigh over 3500 pounds? how many different models weigh over 3500 pounds ?
how many models have either of these features? how many models have either of these features ?
show me the distinct list of their model names! show me the distinct list of their model names !
show the years and weight of all the cars! show the years and weight of all the cars !
show distinct years that the cars that weigh more than 3000 were produced! show distinct years that the cars that weigh more than 3000 were produced !
how about the different years for those that weight less than 4000. how about the different years for those that weight less than 4000 .
what are the distinct years, for the cars that are both? what are the distinct years , for the cars that are both ?
what is the lowest accelerate for any car? what is the lowest accelerate for any car ?
how about the highest? how about the highest ?
what is the id, make and model of this car? what is the id , make and model of thi

list years of work of conductors. list years of work of conductors .
list the name of the conductor with the most years of work. list the name of the conductor with the most years of work .
show the names of conductors. show the names of conductors .
please list these names with the orchestras they have conducted. please list these names with the orchestras they have conducted .
please show the names of the conductors. please show the names of the conductors .
which of them have conducted more than one orchestras? which of them have conducted more than one orchestras ?
please show the names of the conductors. please show the names of the conductors .
who has conducted the most orchestras? who has conducted the most orchestras ?
please show the names of the conductors. please show the names of the conductors .
who has conducted orchestras founded after 2008? who has conducted orchestras founded after 2008 ?
please show the record companies without duplicates. please show the record comp

which of them is the longest? which of them is the longest ?
show the document id for all paragraphs. show the document id for all paragraphs .
for each of them, count the number of paragraphs. for each of them , count the number of paragraphs .
sort them by these counts. sort them by these counts .
which has the fewest? which has the fewest ?
show the document id for all paragraphs. show the document id for all paragraphs .
for each of these ids, show the number of paragraphs. for each of these ids , show the number of paragraphs .
which of them have between 1 and 2 paragraphs? which of them have between 1 and 2 paragraphs ?
show information for all singers. show information for all singers .
how many are there? how many are there ?
show names of all singers. show names of all singers .
for each of them, also show country and age. for each of them , also show country and age .
list the results in descending order of their age. list the results in descending order of their age .
show a

what are their names and membership levels? what are their names and membership levels ?
sort by their age from old to young. sort by their age from old to young .
how many museums are in the table? how many museums are in the table ?
what is the average number of staff working in these museums? what is the average number of staff working in these museums ?
find the name of the museum that has the most staff members. find the name of the museum that has the most staff members .
what is its id? what is its id ?
find the names of museums opened before 2010. find the names of museums opened before 2010 .
how about after? how about after ?
find the minimum number of staff of those museums. find the minimum number of staff of those museums .
find the names of museums which have more staff than that. find the names of museums which have more staff than that .
find the names of visitors who visited and who have some museum visit records. find the names of visitors who visited and who have som

sort all the teachers by their age. sort all the teachers by their age .
which teacher is the youngest? which teacher is the youngest ?
what is his or her hometown? what is his or her hometown ?
what is the hometown of each teacher? what is the hometown of each teacher ?
can you order teachers by their hometowns? can you order teachers by their hometowns ?
show the number of teachers from each hometown. show the number of teachers from each hometown .
how many teachers are there? how many teachers are there ?
how many are there for each hometown? how many are there for each hometown ?
find the most common hometown of teachers. find the most common hometown of teachers .
how many teachers are there? how many teachers are there ?
how many teachers are there for each hometown? how many teachers are there for each hometown ?
which hometowns are shared by at least two teachers? which hometowns are shared by at least two teachers ?
find all the course arrangements. find all the course arrang

how many of them live in connecticut? how many of them live in connecticut ?
what about in indiana? what about in indiana ?
among all the professionals, which have done treatment only 1 treatments? among all the professionals , which have done treatment only 1 treatments ?
list the id, last name and cell phone of the professionals who live in the state of indiana or have done treatment on more than 2 treatments? list the id , last name and cell phone of the professionals who live in the state of indiana or have done treatment on more than 2 treatments ?
show the dog name and total cost of each dog. show the dog name and total cost of each dog .
which of them cost over 500? which of them cost over 500 ?
what about more than 1000? what about more than 1000 ?
list all the names of both professionals and owners. list all the names of both professionals and owners .
what about the dog names? what about the dog names ?
which human names are not used as dog names. which human names are not us

### EditSQL

In [41]:
editsql_sparc_predictions = read_predictions(editsql_sparc_path)

422 1203
422
1203


In [43]:
editsql_sparc_predictions[0]

{'example_id': 1,
 'interaction': [{'interaction_id': 1,
   'turn_id': 1,
   'question': 'What are all the airlines ?',
   'ground_truth': 'SELECT * FROM AIRLINES',
   'sql': 'select * from airlines',
   'exact': True},
  {'interaction_id': 1,
   'turn_id': 2,
   'question': 'Of these , which is Jetblue Airways ?',
   'ground_truth': 'SELECT * FROM AIRLINES WHERE Airline  =  "JetBlue Airways"',
   'sql': 'select * from airlines where Airline = 1',
   'exact': True},
  {'interaction_id': 1,
   'turn_id': 3,
   'question': 'What is the country corresponding it ?',
   'ground_truth': 'SELECT Country FROM AIRLINES WHERE Airline  =  "JetBlue Airways"',
   'sql': 'select T2.Country from airlines as T1 join airports as T2 where T1.Airline = 1',
   'exact': 0}]}

In [45]:
measure_accuracy(editsql_sparc_predictions)

Question Match: 0.471322
Interaction Match: 0.293839


In [47]:
measure_context_independent_accuracy(editsql_sparc_predictions, sparc_dev, sparc_annotation_df)

what are all the airlines? what are all the airlines ?
of these, which is jetblue airways? of these , which is jetblue airways ?
what is the country corresponding it? what is the country corresponding it ?
what are all the abbreviations? what are all the abbreviations ?
what is the abbreviation for jetblue airways? what is the abbreviation for jetblue airways ?
list all airline names and their abbreviations. list all airline names and their abbreviations .
of these, list only those in the usa. of these , list only those in the usa .
find all information about the airport in the city anthony. find all information about the airport in the city anthony .
what is the airport code and airport name? what is the airport code and airport name ?
what are all the airlines? what are all the airlines ?
how many are there? how many are there ?
what are all the airports? what are all the airports ?
how many are there? how many are there ?
what are all the flights? what are all the flights ?
how many

what type are they? what type are they ?
also, how much do they weigh? also , how much do they weigh ?
what are ids of every pet? what are ids of every pet ?
for those, which refer to pets older than 1? for those , which refer to pets older than 1 ?
also, how much do they weigh? also , how much do they weigh ?
what are the different pet types? what are the different pet types ?
for each of those, what is the maximum age? for each of those , what is the maximum age ?
what about the average age? what about the average age ?
what information is there on pets? what information is there on pets ?
for each pet type, what is the average weight? for each pet type , what is the average weight ?
which students have pets? which students have pets ?
what are the different first names? what are the different first names ?
also, what are their ages? also , what are their ages ?
which students have pets? which students have pets ?
of those, whose last name is smith? of those , whose last name is smit

what are the names of the cities in those countries? what are the names of the cities in those countries ?
what are the countries in asia where chinese is spoken? what are the countries in asia where chinese is spoken ?
of those, which have chinese as the official language? of those , which have chinese as the official language ?
what are the different cities in those countries? what are the different cities in those countries ?
which country has the smallest population? which country has the smallest population ?
what is its name, independence year, and surface area? what is its name , independence year , and surface area ?
what is the country with the largest surface area? what is the country with the largest surface area ?
what is its population, name and leader? what is its population , name and leader ?
what are the names of the countries? what are the names of the countries ?
which of them speak at least three languages? which of them speak at least three languages ?
also, how ma

how many is that? how many is that ?
list the name and date of the battle that has lost the ship named 'lettice' list the name and date of the battle that has lost the ship named 'lettice '
same for 'hms atlanta', please. same for 'hms atlanta ' , please .
can you intersect those? can you intersect those ?
which battle ids and names lost ships in the 'english channel'? which battle ids and names lost ships in the 'english channel ' ?
which did not? which did not ?
for these ones, show their names, results, and bulgarian commanders. for these ones , show their names , results , and bulgarian commanders .
how many continents are there? how many continents are there ?
show all their ids and names! show all their ids and names !
what countries do they each have? what countries do they each have ?
count how many there are for each continent name and id! count how many there are for each continent name and id !
what are the names and ids of the different car makers? what are the names and id

list all the different models from these that have makes that are lighter than 3500! list all the different models from these that have makes that are lighter than 3500 !
what are all the names of countries? what are all the names of countries ?
which ones have at least one car maker? which ones have at least one car maker ?
show me all of car names other than these! show me all of car names other than these !
show all the car maker id, names, the models and makes they produce! show all the car maker id , names , the models and makes they produce !
how many model does each car maker produce? list the id and maker! how many model does each car maker produce ? list the id and maker !
show me those that produce at least 2! show me those that produce at least 2 !
for each of these, only show those that have more than 3 car makes! for each of these , only show those that have more than 3 car makes !
how many matches are there? how many matches are there ?
what was the maximum length in minu

show the document name for all documents. show the document name for all documents .
also show their description. also show their description .
which one has description with the letter 'w' in it? which one has description with the letter 'w ' in it ?
show its name and template id. show its name and template id .
show the the document name for all documents. show the the document name for all documents .
show the document id for the one named "robbin cv"? show the document id for the one named `` robbin cv '' ?
also show its template id and description. also show its template id and description .
show the template id for all documents. show the template id for all documents .
show all distinct results. show all distinct results .
how many are there? how many are there ?
show the document ids. show the document ids .
also show their template id and template type code. also show their template id and template type code .
which documents are using the template with type code 'ppt'? which 

show age for all singers. show age for all singers .
what about age for those from france? what about age for those from france ?
what is the average, minimum, and maximum age for those? what is the average , minimum , and maximum age for those ?
show the names of all songs. show the names of all songs .
also show the release year for each of them. also show the release year for each of them .
order the result by the age of their singers. order the result by the age of their singers .
what is the result for the song by the yongest? what is the result for the song by the yongest ?
show country for all singers. show country for all singers .
only show the results for those above 20. only show the results for those above 20 .
what are all distinct countries for them? what are all distinct countries for them ?
show the country for all singers. show the country for all singers .
for each of them, what is the number of singers? for each of them , what is the number of singers ?
what is the a

what is the total spent on all visits? what is the total spent on all visits ?
find the name of the visitor who has spent the most money for his or her visits. find the name of the visitor who has spent the most money for his or her visits .
what are his id and membership level? what are his id and membership level ?
show the id and name of all museums. show the id and name of all museums .
show the number of tickets sold for each of them. show the number of tickets sold for each of them .
which one was visited the most times? which one was visited the most times ?
how many museums are there? how many museums are there ?
how many of them have not had any visitors yet? how many of them have not had any visitors yet ?
what are their names? what are their names ?
how many tickets were sold from all of the museums? how many tickets were sold from all of the museums ?
find the id of the visitor who bought the most tickets at once. find the id of the visitor who bought the most tickets at on

also provide vote ids. also provide vote ids .
also show phone numbers. also show phone numbers .
show all area codes. show all area codes .
what is the largest one? what is the largest one ?
how about smallest one? how about smallest one ?
give me both. give me both .
show all votes. show all votes .
which ones are from the state 'ca'? which ones are from the state 'ca ' ?
among those, can you give me the last date created? among those , can you give me the last date created ?
show all contestants. show all contestants .
what are their names? what are their names ?
can you not include the name 'jessie alloway'? can you not include the name 'jessie alloway ' ?
show all votes? show all votes ?
among these, what is the list of states? among these , what is the list of states ?
what are the distinct states and create time? what are the distinct states and create time ?
show all the contestants. show all the contestants .
show all the votes. show all the votes .
can you show contestants wi

show the dog name and total cost of each dog. show the dog name and total cost of each dog .
which of them cost over 500? which of them cost over 500 ?
what about more than 1000? what about more than 1000 ?
list all the names of both professionals and owners. list all the names of both professionals and owners .
what about the dog names? what about the dog names ?
which human names are not used as dog names. which human names are not used as dog names .
show the professional first names and the corresponding treatment type codes for the treatment. show the professional first names and the corresponding treatment type codes for the treatment .
what about each professional's id, role, email and the treatment type code? what about each professional 's id , role , email and the treatment type code ?
which professional did not provide any treatment? keep the id, role and email. which professional did not provide any treatment ? keep the id , role and email .
show the number of dogs for each

In [108]:
## Measure turn

def measure_interaction_match_pattern(predictions: List[Dict], source_examples: List[Dict]):
    pattern_count = dict()
    interaction_match_count = 0
    for p in predictions:
        example_id = p['example_id']
        has_error = False
        for turn in p['interaction']:
            if turn['exact'] == 1 or turn['exact'] is True:
                pass
            else:
                has_error = True
                break
        if has_error:
            continue
            
        interaction_match_count += 1
        
        # Turn statistics
        source = source_examples[example_id - 1]
        hardness = list()
        for turn in source['interaction']:
            h = eval_hardness(turn['sql'])
            hardness.append(h)
        key = "-".join(hardness)
        if key not in pattern_count:
            pattern_count[key] = 0
        pattern_count[key] += 1
    print("Interaction Match Count: ", interaction_match_count)
    return pattern_count

In [109]:
measure_interaction_match_pattern(duorat_sparc_predictions, sparc_dev)

Interaction Match Count:  146


{'easy-easy': 36,
 'medium-medium': 12,
 'easy-medium': 22,
 'medium-extra': 4,
 'easy-easy-easy': 9,
 'easy-easy-medium': 20,
 'easy-hard': 10,
 'medium-hard': 3,
 'easy-easy-hard': 1,
 'easy-easy-easy-medium': 1,
 'medium-medium-medium-easy': 1,
 'easy-medium-medium-extra': 1,
 'easy-medium-easy': 3,
 'easy-medium-hard': 2,
 'medium-easy': 1,
 'easy-medium-medium': 6,
 'medium-medium-medium': 2,
 'easy-medium-medium-hard': 1,
 'medium-extra-extra': 1,
 'medium-medium-extra': 1,
 'medium-easy-hard-hard': 1,
 'easy-easy-extra': 1,
 'easy-extra-hard': 1,
 'easy-easy-hard-hard': 1,
 'easy-easy-medium-medium': 2,
 'extra-extra': 1,
 'hard-hard': 1,
 'easy-easy-easy-hard': 1}

In [115]:
duorat_sparc_predictions

[{'example_id': 1,
  'interaction': [{'interaction_id': 1,
    'turn_id': 1,
    'question': ' [SEP] What are all the airlines?',
    'groundtruth': 'SELECT * FROM AIRLINES',
    'sql': 'SELECT airlines.Airline FROM airlines',
    'exact': 0},
   {'interaction_id': 1,
    'turn_id': 2,
    'question': ' [SEP] Of these, which is Jetblue Airways? [SEP] What are all the airlines?',
    'groundtruth': 'SELECT * FROM AIRLINES WHERE Airline  =  "JetBlue Airways"',
    'sql': 'SELECT * FROM airlines WHERE airlines.Airline = " Jetblue Airways "',
    'exact': True},
   {'interaction_id': 1,
    'turn_id': 3,
    'question': ' [SEP] What is the country corresponding it? [SEP] Of these, which is Jetblue Airways? [SEP] What are all the airlines?',
    'groundtruth': 'SELECT Country FROM AIRLINES WHERE Airline  =  "JetBlue Airways"',
    'sql': 'SELECT airlines.Country FROM airlines WHERE airlines.Airline = " Jetblue Airways "',
    'exact': True}]},
 {'example_id': 2,
  'interaction': [{'interact

In [117]:
def evaluate_easy(sparc_dev):
    for ex in sparc_dev:
        for turn in ex['interaction']:
            h = eval_hardness(turn['sql'])
            if h == "easy":
                print(turn['query'])

In [118]:
evaluate_easy(sparc_dev)

SELECT * FROM AIRLINES
SELECT * FROM AIRLINES WHERE Airline  =  "JetBlue Airways"
SELECT Country FROM AIRLINES WHERE Airline  =  "JetBlue Airways"
SELECT Abbreviation FROM AIRLINES
SELECT Abbreviation FROM AIRLINES WHERE Airline  =  "JetBlue Airways"
SELECT * FROM AIRPORTS WHERE city  =  "Anthony"
SELECT * FROM AIRLINES
SELECT count(*) FROM AIRLINES
SELECT * FROM AIRPORTS
SELECT count(*) FROM AIRPORTS
SELECT * FROM FLIGHTS
SELECT count(*) FROM FLIGHTS
SELECT Airline FROM AIRLINES
SELECT Airline FROM AIRLINES WHERE Abbreviation  =  "UAL"
SELECT airline FROM AIRLINES WHERE Country  =  "USA"
SELECT count(*) FROM AIRLINES WHERE Country  =  "USA"
SELECT AirportName FROM AIRPORTS
SELECT AirportName FROM AIRPORTS WHERE AirportCode  =  "AKO"
SELECT AirportName FROM AIRPORTS
SELECT AirportName FROM AIRPORTS WHERE City = "Aberdeen"
SELECT * FROM FLIGHTS WHERE SourceAirport  =  "APG"
SELECT count(*) FROM FLIGHTS WHERE SourceAirport  =  "APG"
SELECT * FROM FLIGHTS WHERE DestAirport  =  "ATO"
SELEC